In [38]:
import xarray as xr
import xesmf as xe
import gfdl_utils as gu
import glob

#### Read model

In [27]:
pathDict = {'pp':'/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp',
            'out':'ts'}
pathDict['ppname']='ocean_monthly_z'
pathDict['time']='201001-201412'
pathDict['local']='monthly/5yr'

In [33]:
gu.core.issue_dmget(paths)

0

In [29]:
dsm = xr.Dataset()
for variable in ['thetao','so']:
    pathDict['add']=variable
    paths = gu.core.get_pathspp(**pathDict)
    dsm[variable] = xr.open_mfdataset(paths)[variable]
dsm

KeyboardInterrupt: 

In [34]:
pathstatic = gu.core.get_pathstatic(pathDict['pp'],pathDict['ppname'])
grid = xr.open_dataset(pathstatic)

#### Read observations

In [36]:
basedir = '/work/gam/EN4/EN.4.2.2/analysis/'
filename = '*.nc'

In [37]:
dso = xr.open_mfdataset(basedir+filename)

#### Regrid model to observations

In [22]:
# Subselect model
regridder = xe.Regridder(grid, dso, "bilinear", locstream_out=True)
tmp = regridder(dsm.squeeze())
dsmo = tmp.sel({'z_l':dso['depth']},method='nearest').drop(['z_l','time'])

<xarray.Dataset>
Dimensions:                          (depth: 42, lat: 173, lon: 360, time: 60,
                                      bnds: 2)
Coordinates:
  * depth                            (depth) float32 5.022 15.08 ... 5.35e+03
  * lat                              (lat) float32 -83.0 -82.0 ... 88.0 89.0
  * lon                              (lon) float32 1.0 2.0 3.0 ... 359.0 360.0
  * time                             (time) datetime64[ns] 2010-01-16T12:00:0...
Dimensions without coordinates: bnds
Data variables:
    temperature                      (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    salinity                         (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    temperature_uncertainty          (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    salinity_uncertainty             (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    temperature_observation_weights  (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    salinity_observation_weights     (time, depth, lat, lon) float32 dask.array<chunksize=(1, 42, 173, 360), meta=np.ndarray>
    time_bnds                        (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    depth_bnds                       (time, depth, bnds) float32 dask.array<chunksize=(1, 42, 2), meta=np.ndarray>
Attributes: (12/22)
    Conventions:            CF-1.0
    title:                  Temperature and salinity analysis
    DSD_entry_id:           UKMO-L4UHFnd-GLOB-v01
    references:             Website and paper: https://www.metoffice.gov.uk/h...
    institution:            UK Met Office
    contact:                Rachel Killick - rachel.killick@metoffice.gov.uk
    ...                     ...
    southernmost_latitude:  -90.5
    northernmost_latitude:  89.5
    westernmost_longitude:  0.5
    easternmost_longitude:  362.5
    file_quality_index:     0
    licence:                EN4 is distributed under the Non Commercial Gover...